# Setup

In [1]:
if (!require("xfun")) install.packages("xfun")
pkgs = c(
  'forecast', 'vars', 'urca', 'MLmetrics', 
  'lubridate', 'tsbox', 'timetk',
  'extrafont', 'patchwork', 'hrbrthemes', 'ggthemes', 'ggsci', 'scales', 
  'tidyverse', 'readxl', 'writexl'
)
xfun::pkg_attach2(pkgs, message = FALSE)

options(
  repr.plot.width=10, 
  repr.plot.height=6, 
  repr.plot.res = 300,
  repr.matrix.max.rows = 10,
  repr.matrix.max.cols = Inf
)

loadfonts(device = "win", quiet = TRUE)

Loading required package: xfun


Attaching package: 'xfun'


The following objects are masked from 'package:base':

    attr, isFALSE




# Clean Data
- import data from excel files
- make wide and long format
- make annaul and monthly frequency
- production = export + consumption

## Production Data

In [2]:
production <- read_excel('../data/raw/rice_production.xlsx', 'all')

production_y <- production %>% 
  group_by(year, rice_type) %>%
  summarize(
    area_plant = sum(area_plant),
    area_harvest = sum(area_harvest),
    production = sum(production)*0.66 # convert to milled rice
  ) %>%
  mutate(
    yield_plant = production*1000/area_plant,
    yield_harvest = production*1000/area_harvest
  ) %>% ungroup()

production_wide_y <- production_y %>%
  pivot_wider(
    year,
    rice_type,
    values_from = production
  )

names(production_wide_y) <- c(
  'year_th', 'q_pathum', 'q_hommali', 'q_white', 'q_glutinous'
)

production_wide_y %>% tail()

`summarise()` regrouping output by 'year' (override with `.groups` argument)



year_th,q_pathum,q_hommali,q_white,q_glutinous
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2557,1037038.2,5693006,12490115,5042944
2558,737410.7,5764920,9656676,4708132
2559,664396.3,5796894,7492643,4143932
2560,919589.2,6125247,9470159,4510751
2561,865119.4,5982158,10568502,4297496
2562,1140052.3,5860384,10237382,4111937


## Export Data

In [3]:
# Read and join meta data
export <- read_rds('../data/raw/export_1006_master.rds')
export_meta <- read_excel('../data/raw/meta_hs1006_th.xlsx')
export_joined <- export %>% left_join(export_meta, by = c('hscode' = 'hscode'))


# Yearly data
export_y <- export_joined %>% 
  group_by(year_th, varities) %>%
  summarize(vol = sum(vol)) %>% ungroup()

export_wide_y <- export_y %>%
  pivot_wider(
    year_th,
    varities,
    values_from = vol
  )

names(export_wide_y) <- c(
  'year_th', 'ex_glutinous', 'ex_white', 
  'ex_hommali', 'ex_pathum', 'ex_colored'
)


# Monthly data
export_m <- export_joined %>% 
  group_by(year_th, month, varities) %>%
  summarize(vol = sum(vol)) %>% ungroup()

export_wide_m <- export_m %>%
  pivot_wider(
    c(year_th, month),
    varities,
    values_from = vol,
    values_fill = 0
  ) %>% filter(year_th >= 2545)

names(export_wide_m) <- c(
  'year_th', 'month',
  'ex_glutinous','ex_white','ex_hommali','ex_pathum','ex_colored'
)

head(export_wide_m)

`summarise()` regrouping output by 'year_th' (override with `.groups` argument)

`summarise()` regrouping output by 'year_th', 'month' (override with `.groups` argument)



year_th,month,ex_glutinous,ex_white,ex_hommali,ex_pathum,ex_colored
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2545,01,13054.26,665723.1,11889.38,0,0
2545,02,18478.14,451630.1,27416.01,0,0
2545,03,14681.43,547161.8,41099.09,0,0
2545,04,20208.44,525740.0,17749.08,0,0
2545,05,18239.31,436294.4,46268.54,0,0
2545,06,15290.16,367622.9,144787.19,0,0


## Domestic Consumption

In [4]:
rice_m <- export_wide_m %>% 
  mutate(date = as.Date(paste0(year_th-543, '-', month, '-1')))

rice_y <- export_wide_y %>% 
  left_join(production_wide_y, by = 'year_th') %>%
  mutate(dom_hommali = q_hommali - ex_hommali,
         dom_pathum = q_pathum - ex_pathum,
         dom_white = q_white - ex_white,
         dom_glutinous = q_glutinous - ex_glutinous
        ) %>%
  filter(year_th >= 2545)

rice_m %>% tail
rice_y %>% tail

year_th,month,ex_glutinous,ex_white,ex_hommali,ex_pathum,ex_colored,date
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>
2562,07,7692.439,394990.9,106111.0,39387.40,532.370,2019-07-01
2562,08,5470.075,283164.5,105461.6,53798.72,580.804,2019-08-01
2562,09,3753.291,429410.5,102659.0,40384.34,466.939,2019-09-01
2562,10,4615.151,450419.0,95889.8,55231.47,762.916,2019-10-01
2562,11,10173.654,391762.7,139231.7,35749.23,778.133,2019-11-01
2562,12,12112.880,280986.1,138271.8,32232.51,878.974,2019-12-01


year_th,ex_glutinous,ex_white,ex_hommali,ex_pathum,ex_colored,q_pathum,q_hommali,q_white,q_glutinous,dom_hommali,dom_pathum,dom_white,dom_glutinous
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2557,334083.7,8604516,1869673,161071.4,NA,1037038.2,5693006,12490115,5042944,3823333,875966.8,3885598.3,4708860
2558,264432.5,7419719,1987232,124397.8,NA,737410.7,5764920,9656676,4708132,3777689,613013.0,2236957.9,4443700
2559,337273.7,7071017,2367685,131892.0,NA,664396.3,5796894,7492643,4143932,3429208,532504.2,421625.6,3806658
2560,256136.8,8711069,2308789,386209.9,12126.20,919589.2,6125247,9470159,4510751,3816458,533379.4,759089.1,4254614
2561,223977.8,8876723,1665658,454415.1,11402.34,865119.4,5982158,10568502,4297496,4316500,410704.3,1691778.8,4073518
2562,130604.2,5515012,1410811,514007.5,10077.93,1140052.3,5860384,10237382,4111937,4449573,626044.8,4722370.4,3981332


# Save

In [5]:
# save in csv format
rice_y %>% write_excel_csv('../data/clean/rice_y.csv')
rice_m %>% write_excel_csv('../data/clean/rice_m.csv')

### Read

In [ ]:
rice_a = read_csv('../data/rice_a.csv') 
rice_m = read_csv('../data/rice_m.csv')
rice_m_ts = rice_m %>% tk_ts(start = 2545-543, frequency = 12)

price = read_csv("../data/price_rice.csv") %>%
  filter(year(date) < 2020)
price_ts = price %>% tk_ts(start = 2000, frequency = 12)

exo = read_csv("../data/wdi_exo.csv")
exo_ts = exo %>% tk_ts(start = 2002, frequency = 1)

data = ts_c(rice_m_ts, price_ts, exo_ts) %>% window(start = 2003)

data = data %>% na.locf()

In [ ]:
data %>% head

## Data Visualization

In [ ]:
p_hommali = ggplot(rice_a, aes(x=year_th)) +
  geom_line( aes(y=dom_hommali/1e6, color = 'Domestic')) + 
  geom_line( aes(y=ex_hommali/1e6, color = 'Export')) + 
  scale_colour_manual("",
                      label = c('บริโภคในประเทศ', 'ส่งออก'),
                      breaks = c("Domestic", "Export"),
                      values = c("Domestic"="#0b53c1", 
                                 "Export"="#ff0055")
                     ) +
  labs(title = 'ข้าวหอมมะลิ',
       x =  "ปี",
       y = 'ล้านตันข้าวสาร'
      ) +
  expand_limits(y = 0) +
  scale_y_continuous(label = comma) +
  theme_ipsum(
    base_size = 16,
    base_family = "Athiti Light",
    axis_title_size = 16
  ) +
  theme(
    legend.position = c(1, 1),
    legend.direction = 'horizontal',
    legend.justification = 'right',
    legend.title = element_blank()
  )

p_hommali
ggsave('../figures/plot_hommali.png', width = 8, height = 4)

In [ ]:
p_pathum = ggplot(rice_a, aes(x=year_th)) +
  geom_line( aes(y=dom_pathum/1e6, color = 'Domestic')) + 
  geom_line( aes(y=ex_pathum/1e6, color = 'Export')) + 
  scale_colour_manual("",
                      label = c('บริโภคในประเทศ', 'ส่งออก'),
                      breaks = c("Domestic", "Export"),
                      values = c("Domestic"="#0b53c1", 
                                 "Export"="#ff0055")
                     ) +
  labs(title = 'ข้าวหอมปทุม',
       x =  "ปี",
       y = 'ล้านตันข้าวสาร'
      ) +
  expand_limits(y = 0) +
  scale_y_continuous(label = comma) +
  theme_ipsum(
    base_size = 16,
    base_family = "Athiti Light",
    axis_title_size = 16
  ) +
  theme(
    legend.position = c(1, 1),
    legend.direction = 'horizontal',
    legend.justification = 'right',
    legend.title = element_blank()
  )

p_pathum
ggsave('../figures/plot_pathum.png', width = 8, height = 4)

In [ ]:
p_white = ggplot(rice_a, aes(x=year_th)) +
  geom_line( aes(y=dom_white/1e6, color = 'Domestic')) + 
  geom_line( aes(y=ex_white/1e6, color = 'Export')) + 
  scale_colour_manual("",
                      label = c('บริโภคในประเทศ', 'ส่งออก'),
                      breaks = c("Domestic", "Export"),
                      values = c("Domestic"="#0b53c1", 
                                 "Export"="#ff0055")
                     ) +
  labs(title = 'ข้าวเจ้าขาว',
       x =  "ปี",
       y = 'ล้านตันข้าวสาร'
      ) +
  expand_limits(y = 0) +
  scale_y_continuous(label = comma) +
  theme_ipsum(
    base_size = 16,
    base_family = "Athiti Light",
    axis_title_size = 16
  ) +
  theme(
    legend.position = c(1, 1),
    legend.direction = 'horizontal',
    legend.justification = 'right',
    legend.title = element_blank()
  )

p_white
ggsave('../figures/plot_white.png', width = 8, height = 4)

In [ ]:
p_glutinous = ggplot(rice_a, aes(x=year_th)) +
  geom_line( aes(y=dom_glutinous/1e6, color = 'Domestic')) + 
  geom_line( aes(y=ex_glutinous/1e6, color = 'Export')) + 
  scale_colour_manual("",
                      label = c('บริโภคในประเทศ', 'ส่งออก'),
                      breaks = c("Domestic", "Export"),
                      values = c("Domestic"="#0b53c1", 
                                 "Export"="#ff0055")
                     ) +
  labs(title = 'ข้าวเหนียว',
       x =  "ปี",
       y = 'ล้านตันข้าวสาร'
      ) +
  expand_limits(y = 0) +
  scale_y_continuous(label = comma) +
  theme_ipsum(
    base_size = 16,
    base_family = "Athiti Light",
    axis_title_size = 16
  ) +
  theme(
    legend.position = c(1, 1),
    legend.direction = 'horizontal',
    legend.justification = 'right',
    legend.title = element_blank()
  )

p_glutinous
ggsave('../figures/plot_glutinous.png', width = 8, height = 4)

In [ ]:
ggplot(rice_m, aes(x=date)) +
  geom_line( aes(y=ex_hommali/1e6, color = 'ข้าวหอมมะลิ')) +
  geom_line( aes(y=ex_pathum/1e6, color = 'ข้าวหอมปทุม')) +
  geom_line( aes(y=ex_white/1e6, color = 'ข้าวเจ้าขาว')) +
  geom_line( aes(y=ex_glutinous/1e6, color = 'ข้าวเหนียว')) +
  scale_color_d3() +
  expand_limits(y = 0) +
  scale_y_continuous(label = comma) +
  labs(title = 'ปริมาณส่งออกข้าวสารของไทย',
       x =  "ปี",
       y = 'ล้านตันข้าวสาร'
      ) +
  theme_ipsum(
    base_size = 16,
    base_family = "Athiti Light",
    axis_title_size = 16
  ) +
  theme(
    legend.position = c(1, 1),
    legend.direction = 'horizontal',
    legend.justification = 'right',
    legend.title = element_blank()
  ) +
  scale_x_date(date_breaks = '2 years',
               labels = function(x) year(x)+543
               )

ggsave('../figures/plot_export_m.png', width = 10, height = 7)

## Unit Root Test

In [ ]:
get_objname = function(x) deparse(substitute(x))

ur_adf = function(y, n_diff = 0, varname = NULL, ...) {
  
  varname = if (is.null(varname)) deparse(substitute(y)) else varname
  y = if(n_diff == 0) y else diff(y, n_diff)
  
  ur.trend = ur.df(y, type='trend', selectlags = "AIC", ...)
  ur.drift = ur.df(y, type='drift', selectlags = "AIC", ...)
  ur.none  = ur.df(y, type='none', selectlags = "AIC", ...)

  tstat.trend = ur.trend@teststat
  tstat.drift = ur.drift@teststat
  tstat.none  = ur.none@teststat

  cv.trend = ur.trend@cval
  cv.drift = ur.drift@cval
  cv.none  = ur.none@cval

  df_test = rbind(
    cbind(t(tstat.trend), cv.trend),
    cbind(t(tstat.drift), cv.drift),
    cbind(t(tstat.none) , cv.none)
  ) %>% 
  as.data.frame() %>%
  rownames_to_column("hypo") %>%
  mutate(
    result = ifelse(abs(statistic) >= abs(`5pct`), 'Reject', 'Accept'),
    variable = varname,
    level = paste0('d', n_diff),
    statistic = round(statistic, 2),
  ) %>%
  filter(str_starts(hypo, 'tau')) %>%
  select(variable, hypo, level, everything())
  
  return(df_test)
}

In [ ]:
ex_hommali = data[, 'ex_hommali'] %>% log()
ex_white = data[, 'ex_white'] %>% log()
p_w5_th = data[, 'p_w5_th'] %>% log()
p_w5_vn = data[, 'p_w5_vn'] %>% log()
p_h100_th = data[, 'p_h100_th'] %>% log()
gdppc = data[, 'gdppc'] %>% log()
pop = data[, 'pop'] %>% log()

vars_list = list(
  ex_hommali = ex_hommali, 
  ex_white = ex_white, 
  p_w5_th = p_w5_th, 
  p_w5_vn = p_w5_vn, 
  p_h100_th = p_h100_th,
  gdppc = gdppc, 
  pop = pop
)

varnames = rep(names(vars_list), 2)

params = expand_grid(varnames, c(0,1)) %>%
  set_names(c('names', 'ndiff')) %>%
  mutate(ser = vars_list[names])

pmap(list(params$ser, params$ndiff, params$names), 
     function(first, second, third) {
       ur_adf(first, second, third)
     }) %>% 
  reduce(rbind) %>% 
  write_excel_csv('../results/adf_table.csv') %>%
  print()

# rbind(
#   ur_adf(ex_hommali, 0)   , ur_adf(ex_hommali, 1),
#   ur_adf(ex_white, 0)     , ur_adf(ex_white, 1),
#   ur_adf(p_w5_th, 0)      , ur_adf(p_w5_th, 1),
#   ur_adf(p_w5_vn, 0)      , ur_adf(p_w5_vn, 1),
#   ur_adf(p_h100_th, 0)    , ur_adf(p_h100_th, 1)
# ) %>% print()

## Define function

In [ ]:
ex_hommali %>% subset(end = 12*14)

In [ ]:
14*12

In [ ]:
ets_vec = Vectorize(ets)
x = ex_hommali %>% subset(end = 120)
params = list(y = x, model = "ZZZ", damped = FALSE)

# ets(ex_hommali %>% subset(end = 120), params)
res = do.call(ets, params)

In [ ]:
# forecast(res, 12)
# ets(ex_hommali %>% subset(end = 120), model = "ZZZ", damped = FALSE) %>% str()

In [ ]:
params = list(model = "ZZZ", damped = FALSE)
c(y = "xx", params)

# Strategy
1. 

In [ ]:
params = list(model = "ZZZ", damped = NULL)

ets_fcast = function(i, data, params, h=1) {
  y = data %>% subset(end = i)
  params$y = y
  fitted = do.call(forecast::ets, params)
  res = fitted %>% forecast(h=h) %>% as.data.frame()
  return(res)
}

rolling_ets = function(data, params, n_initial, h) {
  n_sample = length(data)
  res = n_initial:n_sample %>% 
    map(ets_fcast, data = data, params = params, h = h) %>% 
    reduce(rbind)
  return(res)
}

x = rolling_ets(
  ex_hommali, 
  params = list(model = "AAA", damped = TRUE) , 
  n_initial = 168, h = 1)

x

In [ ]:
# fit function
fets = function(x, model='AAN', damped = FALSE, h=h) {
  forecast::ets(x, model = model, damped = damped) %>% forecast(h=h)
}

fets.auto = function(x, h=h) {
  forecast::ets(x, model = 'ZZZ', damped = NULL) %>% forecast(h=h)
}

farima = function(x, order = c(0,0,0), seasonal = c(0,0,0), h=h) {
  forecast::Arima(x,
                  order = order, 
                  seasonal = seasonal,
                 ) %>% forecast(h=h)
}

farima.auto = function(x, d=0, h=h) {
  forecast::auto.arima(x, d=d) %>% forecast(h=h)
}


# accuracy measurement
rmse = function(data, h = h) {
  data = data %>% select('actual', starts_with('yhat')) %>% exp()
  h = ncol(data) - 1
  for (i in seq(h)) {
    data[, paste0('e', i)] = (
      lead(data[,'actual'], i) - data[, paste0('yhat', i)]
    )^2
  }
  rmse_score = data %>% select(starts_with('e')) %>% 
    as.matrix() %>% mean(na.rm = TRUE) %>% sqrt()
  
  return(rmse_score)
}

mape = function(data, h = h) {
  data = data %>% select('actual', starts_with('yhat')) %>% exp()
  h = ncol(data) - 1
  for (i in seq(h)) {
    data[, paste0('e', i)] = abs(
      lead(data[,'actual'], i) - data[, paste0('yhat', i)]
    )*100 / lead(data[,'actual'], i)
  }
  mape_score = data %>% select(starts_with('e')) %>% 
    as.matrix() %>% mean(na.rm = TRUE)
  
  return(mape_score)
}

eval_ets = function(x, params, h = 2, initial = 11) {
  pmap(
    list(params$model, params$damped), 
    function(first, second) {
      
      # 1 compute forecast error using time series cross validation
      error = tsCV(x, fets, h = h, initial = initial, 
                   model = first, 
                   damped = second
                  ) %>% as.data.frame()
      new_colnames = c('actual', paste0('e', seq(1:h)))
      error = data.frame(x, error) %>% set_names(new_colnames)
      
      # 2 compute predicted (yhat)
      for (i in seq(h)) {
        error[, paste0('yhat', i)] = (
          lead(error[,'actual'], i) - error[, paste0('e', i)]
        )
      }
      
      # 3 compute rmse, mape
      data.frame(
        model = paste(first, second, sep = '-'),
        rmse = rmse(error),
        mape = mape(error)
      ) %>% na.omit()
    }
  ) %>% 
  reduce(bind_rows) %>%
  arrange(rmse)
}

eval_arima = function(x, params, h = 2, initial = 11) {
  pmap(
    list(params$order, params$seasonal), 
    function(first, second) {
      
      # 1 compute forecast error using time series cross validation
      error = tsCV(x, farima, h = h, initial = initial, 
                   order = first, 
                   seasonal = second
                  ) %>% as.data.frame()
      new_colnames = c('actual', paste0('e', seq(1:h)))
      error = data.frame(x, error) %>% set_names(new_colnames)
      
      # 2 compute predicted (yhat)
      for (i in seq(h)) {
        error[, paste0('yhat', i)] = (
          lead(error[,'actual'], i) - error[, paste0('e', i)]
        )
      }
  
      # 3 compute rmse, mape
      data.frame(
        model = paste(paste(first, collapse = ','), 
                      paste(second, collapse = ','), sep = '-'),
        rmse = rmse(error),
        mape = mape(error)
      ) %>% na.omit()
    }
  ) %>% 
  reduce(bind_rows) %>%
  arrange(rmse)
}

## Export

In [ ]:
# ETS params
Z1 = c('A','M')
Z2 = c('N', 'A', 'M')
Z3 = c('N', 'A', 'M')
damped = c(FALSE, TRUE)

ets_params = expand.grid(Z1, Z2, Z3, damped) %>%
  mutate(
    model = paste0(Var1, Var2, Var3),
    damped = Var4
) %>% select(model, damped)

# ARIMA params
p = seq(0,2)
q = seq(0,2)
P = seq(0,1)
Q = seq(0,1)

arima_params = expand.grid(p,q,P,Q) %>% rowwise() %>%
  mutate(
    order = pmap(list(Var1, Var2), function(x, y) c(x, 0, y)),
    seasonal = pmap(list(Var3, Var4), function(x, y) c(x, 0, y)),
  ) %>% select(order, seasonal)

In [ ]:
(length(ex_hommali)-8) * 0.80 / 12
216-36

In [ ]:
h = 1
initial = 180

e.ets.ex_hml = eval_ets(
  ex_hommali %>% log(), params = ets_params, h = h, initial = initial
)
e.ets.ex_wht = eval_ets(
  ex_white %>% log(), params = ets_params, h = h, initial = initial
)

In [ ]:
e.arima.ex_hml = eval_arima(
  ex_hommali %>% log(), params = arima_params, h = h, initial = initial
)
e.arima.ex_wht = eval_arima(
  ex_white %>% log(), params = arima_params, h = h, initial = initial
)

In [ ]:
# acc_ex_hml1 = list(ets = e.ets.ex_hml, e.arima.ex_hml)
# acc_ex_pat1 = list(ets = e.ets.ex_pat, e.arima.ex_pat)
# acc_ex_wht1 = list(ets = e.ets.ex_wht, e.arima.ex_wht)
# acc_ex_glu1 = list(ets = e.ets.ex_glu, e.arima.ex_glu)

In [ ]:
acc_ex_hml = list(ets = e.ets.ex_hml %>% head(3), 
                  arima = e.arima.ex_hml %>% head(3))
acc_ex_wht = list(ets = e.ets.ex_wht %>% head(3), 
                  arima = e.arima.ex_wht %>% head(3))

In [ ]:
acc_ex_hml

In [ ]:
acc_ex_wht

In [ ]:
fets <- function(x, model='AAN', damped = FALSE, h=1) {
  forecast(ets(x, model=model, damped = damped), h=h)
}

farima <- function(x, d, h=1) {
  forecast(auto.arima(x, d=d), h=h)
}

In [ ]:
2016+543

In [ ]:
accuracy(res_hml)

In [ ]:
accuracy_ = function(actual, fitted) {
  error = actual - fitted
  n = length(actual)
  rmse = sqrt(sum(error**2)/n)
  mape = mean(abs(error/actual)*100)
  res = data.frame(rmse, mape)
  return(res)
}

In [ ]:
actual = ex_hommali %>% window(end = c(2016, 12))
fitted_model = ets(actual %>% log(), model = 'MAA', damped = TRUE)
fitted = exp(fitted_model$fitted)
accuracy_(actual, fitted)

In [ ]:
actual = ex_hommali %>% window(end = c(2016, 12))
fitted_model = Arima(actual %>% log(), 
                     order = c(1,0,1), seasonal = c(1,0,1)
                    )
 
fitted = exp(fitted_model$fitted)
accuracy_(actual, fitted)

In [ ]:
actual = ex_white %>% window(end = c(2016, 12))
fitted_model = ets(actual %>% log(), model = 'MMN', damped = FALSE)
fitted = exp(fitted_model$fitted)
accuracy_(actual, fitted)

In [ ]:
actual = ex_white %>% window(end = c(2016, 12))
fitted_model = Arima(actual %>% log(), 
                     order = c(1,0,1), seasonal = c(1,0,1)
                    )
fitted = exp(fitted_model$fitted)
accuracy_(actual, fitted)

In [ ]:
fets(ex_hommali %>% log(), model = 'MAA', damped = TRUE, h = 24) %>% 
  autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'ข้าวหอมมะลิ')
ggsave('../figures/plot_ets_hommali_ex.png', width = 10, height = 5)


fets(ex_white %>% log(), model = 'MMN', damped = FALSE, h = 24) %>% 
  autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'ข้าวเจ้าขาว')
ggsave('../figures/plot_ets_white_ex.png', width = 10, height = 5)

In [ ]:
fets(ex_hommali %>% log(), model = 'MAA', damped = TRUE, h = 24) %>%
  as.data.frame() %>%
  write_excel_csv('../results/export_hml.csv')

fets(ex_white %>% log(), model = 'MMN', damped = FALSE, h = 24) %>%
  as.data.frame() %>%
  write_excel_csv('../results/export_wht.csv')

# farima(ex_pathum %>% log(), 
#        order = c(1,0,1), seasonal = c(1,0,1), h = 24) %>%
#   as.data.frame() %>%
#   write_excel_csv('../results/export_pat.csv')


# farima(ex_glutinous %>% log(), 
#        order = c(2,0,1), 
#        seasonal = c(1,0,1), h = 24) %>%
#   as.data.frame() %>%
#   write_excel_csv('../results/export_glu.csv')

In [ ]:
fets(log(ts_export[,'ex_hommali']), h = 24) %>% autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'Hommali Rice')
ggsave('../figures/plot_ets_exp_hml.png', width = 8, height = 5)

fets(log(ts_export[,'ex_pathum']), h = 24) %>% autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'Pathumtani Rice')
ggsave('../figures/plot_ets_exp_pat.png', width = 8, height = 5)

fets(log(ts_export[,'ex_white']), h = 24) %>% autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'White Rice')
ggsave('../figures/plot_ets_exp_wht.png', width = 8, height = 5)

fets(log(ts_export[,'ex_glutinous']), h = 24) %>% autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'Glutinous Rice')
ggsave('../figures/plot_ets_exp_glu.png', width = 8, height = 5)

## ARIMA

In [ ]:
farima <- function(x, d, h=2) {
  forecast(auto.arima(x, d=d), h=h)
}

In [ ]:
farima(log(data$dom_hommali), d=1, h=2) %>% autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'Hommali Rice')
ggsave('../figures/plot_arima_dom_hml.png', width = 8, height = 4)

farima(log(data$dom_pathum), d=1, h=2) %>% autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'Pathumtani Rice')
ggsave('../figures/plot_arima_dom_pat.png', width = 8, height = 4)

farima(log(data$dom_white), d=1, h=2) %>% autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'White Rice')
ggsave('../figures/plot_arima_dom_wht.png', width = 8, height = 4)

farima(log(data$dom_glutinous), d=1, h=2) %>% autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'Glutinous Rice')
ggsave('../figures/plot_arima_dom_glu.png', width = 8, height = 4)

In [ ]:
farima(log(ts_export[, 'ex_hommali']), d=1, h=24) %>% autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'Hommali Rice')
ggsave('../figures/plot_arima_exp_hml.png', width = 8, height = 4)

farima(log(ts_export[, 'ex_pathum']), d=1, h=24) %>% autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'Pathumtani Rice')
ggsave('../figures/plot_arima_exp_pat.png', width = 8, height = 4)

farima(log(ts_export[, 'ex_white']), d=1, h=24) %>% autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'White Rice')
ggsave('../figures/plot_arima_exp_wht.png', width = 8, height = 4)

farima(log(ts_export[, 'ex_glutinous']), d=1, h=24) %>% autoplot() +
  theme_ipsum_rc(base_size = 14) + 
  labs(subtitle = 'Glutinous Rice')
ggsave('../figures/plot_arima_exp_glu.png', width = 8, height = 4)

In [ ]:
# Compute CV errors for ETS as e1
e1 <- tsCV(log(data$dom_hommali), fets, h=2, initial = 10)
# Compute CV errors for ARIMA as e2
e2 <- tsCV(log(data$dom_hommali), farima, d=1, h=2, initial = 10)

# MSE
# sqrt(mean(e1[10:13,]^2))
# sqrt(mean(e2[10:13,]^2))

# # MAE
# mean(abs(e1[10:13,]))
# mean(abs(e2[10:13,]))

In [ ]:
forecast(ets(log(data$ex_hommali)), h = 2)

In [ ]:
forecast(auto.arima(log(data$dom_glutinous), d = 1), h = 2) %>% autoplot()

## VAR

In [ ]:
gdppc2 = gdppc ** 2

In [ ]:
endo = ts_c(ex_hommali, ex_white, p_h100_th, p_w5_th, p_w5_vn)
exo = ts_c(gdppc, gdppc2, pop)
class(exo)

In [ ]:
# endo[, "ex_hommali"] %>% stats::lag()

In [ ]:
VARselect(endo, exogen = exo)

In [ ]:
ca.jo(endo, ecdet = 'const', K = 2) %>% summary()

In [ ]:
# subset(endo, end = 180)

In [ ]:
var_fitted = VAR(
  endo, 
  p = 2, 
  type = "const", 
  exogen = exo)

In [ ]:
serial.test(vecm_export, lags.pt=12, type="PT.asymptotic")

In [ ]:
vecm_export[['varresult']][['ex_hommali']]

In [ ]:
forecast(vecm_export, h = 1, exogen = exo[216]) %>% autoplot()
# ggsave('../figures/plot_vecm_exp.png', width = 14, height = 8)